In [19]:
import os
import numpy as np
import tensorflow as tf

In [20]:
from models import *

In [48]:
from tfutils import base

In [ ]:
base.

In [21]:
def shallow_bottle(inputs, train = True, norm = True, **kwargs):
    # propagate input targets
    outputs = inputs
#     dropout = 0.5 if train else None
    
    
    # create the encoder
    outputs['conv1'], outputs['conv1_kernel'] = conv(outputs['images'], 
                                                     64, 
                                                     ksize=7, 
                                                     strides=11, 
                                                     padding = 'SAME', 
                                                     layer = 'conv1')
    
    
    # create the decoder
    out_shape = tf.stack([tf.shape(outputs['image'])[0], shape[1], shape[2], shape[3]])
    outputs['deconv1'], outputs['deconv1_kernel'] = conv(outputs['conv1'], 
                                                                         
                                                         ksize=, 5, 1, layer = 'deconv1')
    return outputs, {}

SyntaxError: invalid syntax (<ipython-input-21-bf6d733696f3>, line 20)

In [22]:
tf.nn.conv2d_transpose

<function tensorflow.python.ops.nn_ops.conv2d_transpose>

In [27]:
def conv(inp,
         out_depth,
         ksize=[3,3],
         strides=[1,1,1,1],
         padding='SAME',
         kernel_init='xavier',
         kernel_init_kwargs=None,
         bias=0,
         weight_decay=None,
         activation='relu',
         batch_norm=False,
         name='conv',
         layer = None,
         ):
    with tf.variable_scope(layer):
        # assert out_shape is not None
        if weight_decay is None:
            weight_decay = 0.
        if isinstance(ksize, int):
            ksize = [ksize, ksize]
            
        if isinstance(strides, int):
            strides = [1, strides, strides, 1]            
            
        if kernel_init_kwargs is None:
            kernel_init_kwargs = {}
        in_depth = inp.get_shape().as_list()[-1]

        # weights
        init = initializer(kernel_init, **kernel_init_kwargs)

        kernel = tf.get_variable(initializer=init,
                                shape=[ksize[0], ksize[1], in_depth, out_depth],
                                dtype=tf.float32,
                                regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                                name='weights')
        init = initializer(kind='constant', value=bias)
        biases = tf.get_variable(initializer=init,
                                shape=[out_depth],
                                dtype=tf.float32,
                                regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                                name='bias')
        # ops
        conv = tf.nn.conv2d(inp, kernel,
                            strides=strides,
                            padding=padding)
        output = tf.nn.bias_add(conv, biases, name=name)

        if activation is not None:
            output = getattr(tf.nn, activation)(output, name=activation)
        if batch_norm:
            output = tf.nn.batch_normalization(output, mean=0, variance=1, offset=None,
                                scale=None, variance_epsilon=1e-8, name='batch_norm')
    return output, kernel

In [31]:
def deconv(inp, 
           out_shape, 
           ksize = [3,3],
           strides = [1,1,1,1],
           padding = 'SAME',
           kernel_init = 'xavier',
           kernel_init_kwargs = None,
           bias = 0, 
           weight_decay = None,
           batch_norm = False,
           name = 'deconv', 
           layer = None,
           dilation = 1):

    with tf.variable_scope(layer):
        # assert out_shape is not None
        if weight_decay is None:
            weight_decay = 0.
        if isinstance(ksize, int):
            ksize = [ksize, ksize]
        if isinstance(strides, int):
            strides = [1, strides, strides, 1]   
        if kernel_init_kwargs is None:
            kernel_init_kwargs = {}            

        # define convolutional kernel and bias
        init = initializer(kernel_init, **kernel_init_kwargs)        
        kernel = tf.get_variable(initializer=init,
                                shape=[ksize[0], ksize[1], out_shape[-1], in_depth]        ,
                                dtype=tf.float32,
                                regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                                name='weights')
        init = initializer(kind='constant', value=bias)
        biases = tf.get_variable(initializer=init,
                                shape=[out_depth],
                                dtype=tf.float32,
                                regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                                name='bias')

        # feed into our deconvolution
        deconv = tf.nn.conv2d_transpose(inp, kernel, out_shape, 
                                        strides = strides, 
                                        padding = padding)
        output = tf.nn.bias_add(deconv, biases, name=name)
        if batch_norm:
            output = tf.nn.batch_normalization(output, mean=0, variance=1, offset=None,
                                scale=None, variance_epsilon=1e-8, name='batch_norm')    
    return output, kernel

In [36]:
from keras.models import Model
from keras.layers import Input, Dense

In [47]:
a = Input(shape=(32,), name = 'test')
b = Dense(32, name = 'test2')(a)
c = Dense(32, name = 'test2')()
model = Model(inputs=a, outputs=b)
model.name

'model_7'